In [39]:
from datetime import datetime, timedelta, timezone
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import base64
from email.mime.text import MIMEText
import os

In [70]:

CREDENTIALS_OAUTH = "credentials-oauth.json"
TOKEN_FILE = "token.json"
SCOPES = ['https://www.googleapis.com/auth/calendar', 'https://www.googleapis.com/auth/gmail.readonly', 'https://www.googleapis.com/auth/gmail.send']

In [71]:
def authenticate_user():
    creds = None

    # Load existing credentials from file if available
    if os.path.exists(TOKEN_FILE):
        creds = Credentials.from_authorized_user_file(TOKEN_FILE, SCOPES)

    # If there are no valid credentials, run the OAuth flow
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_OAUTH, SCOPES)
            creds = flow.run_local_server(port=8080)

        # Save the credentials for the next run
        with open(TOKEN_FILE, 'w') as token_file:
            token_file.write(creds.to_json())

    return creds

In [72]:
creds = authenticate_user()
calendar_service = build('calendar', 'v3', credentials=creds)
email_service = build('gmail', 'v1', credentials=creds)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1010614541530-f1qbtoc2jdf93al918lulholfh9lhh93.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=91I71vuI3ZtZF3dbTSwGGNRq8D1aA0&access_type=offline


In [43]:
date_format = "%Y-%m-%d %H:%M:%S"

In [44]:
def dates_to_strings(dates):
    string_tuples = []
    for slot_start, slot_end in dates:
        string_tuples.append((slot_start.strftime(date_format), slot_end.strftime(date_format)))
        
    return string_tuples

In [45]:
def pst_to_utc(pst_datetime):
    """Convert a naive datetime object from PST (UTC-8) to naive UTC."""
    # Define the UTC offset for PST (UTC-8)
    pst_offset = timedelta(hours=8)
    
    # Localize the naive datetime to PST and convert it to UTC
    utc_time = pst_datetime + pst_offset
    
    # Return as naive UTC datetime (without tzinfo)
    return utc_time

def utc_to_pst(utc_datetime):
    """Convert a naive datetime object from UTC to naive PST (UTC-8)."""
    # Define the UTC offset for PST (UTC-8)
    pst_offset = timedelta(hours=-8)
    
    # Localize the naive datetime to UTC and convert it to PST
    pst_time = utc_datetime + pst_offset
    
    # Return as naive PST datetime (without tzinfo)
    return pst_time

In [46]:
print(pst_to_utc(datetime.strptime("2024-11-13 09:00:00", date_format)))
print(pst_to_utc(datetime.strptime("2024-11-13 11:00:00", date_format)))

2024-11-13 17:00:00
2024-11-13 19:00:00


In [11]:
from langchain.tools import BaseTool, StructuredTool, tool

In [ ]:
def get_availability_slots(participant_emails: list[str], meeting_interval_start_date: str, meeting_interval_end_date: str, meeting_duration_minutes: int) -> list[tuple[str]]:
    '''Finds the available time slots that are free for all involved peers / attendees. Will automatically return time slots that meet the meeting duration time and are available for all participants. Takes a list of participant emails. Don't forget to include the email of the organizer! Will return a list of tuples of datetime strings, representing the start and end of each open interval. Will look for slots only in the provided time window. Please provide proper datetime strings in the format YYYY-mm-dd HH:mm:ss for the interval start and end times. Please be thorough and consider all responses when using this data, every single time slot could be critical! If you are asked to choose some of these slots for meetings, please provide what you think would be the most convenient ones.'''

    start_date = datetime.strptime(meeting_interval_start_date, date_format)
    end_date = datetime.strptime(meeting_interval_end_date, date_format)

    start_date = pst_to_utc(start_date)
    end_date = pst_to_utc(end_date)

    # Prepare the request body for free/busy query
    body = {
        "timeMin": start_date.isoformat() + "Z", # Z is for UTC
        "timeMax": end_date.isoformat() + "Z",
        "items": [{"id": email} for email in participant_emails]
    }

    freebusy_result = calendar_service.freebusy().query(body=body).execute()
    #print(freebusy_result)

    # Collect all busy periods for each calendar
    all_busy_periods = []
    for calendar in freebusy_result['calendars'].values():
        for busy_period in calendar.get('busy', []):
            busy_start = datetime.fromisoformat(busy_period['start'][:-1])  # Remove 'Z' for parsing
            busy_end = datetime.fromisoformat(busy_period['end'][:-1])
            all_busy_periods.append((busy_start, busy_end))

    # TODO implement blocking for non-working hours

    # Sort all busy periods and merge overlapping ones
    all_busy_periods.sort()
    merged_busy_periods = []
    for busy_start, busy_end in all_busy_periods:
        if not merged_busy_periods or merged_busy_periods[-1][1] < busy_start:
            merged_busy_periods.append((busy_start, busy_end))
        else:
            merged_busy_periods[-1] = (merged_busy_periods[-1][0], max(merged_busy_periods[-1][1], busy_end))

    # Find available slots between merged busy periods
    available_slots = []
    current_start = start_date
    for busy_start, busy_end in merged_busy_periods:
        if busy_start - current_start >= timedelta(minutes=meeting_duration_minutes):
            available_slots.append((utc_to_pst(current_start), utc_to_pst(busy_start)))
        current_start = busy_end

    # Check if there's room after the last busy period
    if end_date - current_start >= timedelta(minutes=meeting_duration_minutes):
        available_slots.append((utc_to_pst(current_start), utc_to_pst(end_date)))

    # TODO last appended is null for some reason
    return dates_to_strings(available_slots)

In [15]:
slots = get_availability_slots(["sfigueira@scu.edu", "swu12@scu.edu"], "2024-11-13 09:00:00", "2024-11-14 20:00:00", 45)
for s in slots:
    print(s)
print(len(slots))

('2024-11-13 16:00:00', '2024-11-13 17:00:00')
('2024-11-13 20:00:00', '2024-11-13 23:30:00')
('2024-11-13 23:30:00', '2024-11-14 10:00:00')
3


In [186]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
tools = [get_availability_slots]
react_graph = create_react_agent(model=model, tools=tools)

In [188]:
response = react_graph.invoke({"messages": [("user", "I really want to schedule a 40 minute meeting with sfigueira@scu.edu and swu12@scu.edu sometime this week (it is currently november 12, 2024) can you help me? I want to meet between 9 am and no later than 6 pm. I don't want to meet on weekends. My email is aanokh@scu.edu. Help me!")]})

In [189]:
print(response["messages"][3].content)

Here are the available time slots for a 40-minute meeting between you and your colleagues this week:

### Available Time Slots:
1. **November 12, 2024**: 5:00 PM - 5:40 PM
2. **November 13, 2024**: 9:00 AM - 9:40 AM
3. **November 13, 2024**: 4:00 PM - 4:40 PM
4. **November 14, 2024**: 10:00 AM - 10:40 AM
5. **November 15, 2024**: 6:00 PM - 6:40 PM
6. **November 16, 2024**: 2:00 PM - 2:40 PM
7. **November 16, 2024**: 4:00 PM - 4:40 PM
8. **November 17, 2024**: 5:00 PM - 5:40 PM

Please let me know which time slot works best for you!


In [194]:
response["messages"] += ("user", "eh whatever, i dont want to meat with swu anymore, but i need this meeting urgent until the 14th at midnight. can you arrange that?")
response = react_graph.invoke(response)

In [200]:
print(response["messages"][13].content)

Here are the available time slots for a 40-minute meeting with you, sfigueira@scu.edu, and lamlicke@scu.edu before midnight on November 14, 2024:

### Available Time Slots:
1. **November 12, 2024**: 9:00 AM - 9:40 AM
2. **November 12, 2024**: 5:00 PM - 5:40 PM
3. **November 13, 2024**: 4:00 PM - 4:40 PM
4. **November 14, 2024**: 9:00 AM - 9:40 AM
5. **November 14, 2024**: 3:00 PM - 3:40 PM
6. **November 14, 2024**: 6:00 PM - 6:40 PM

Please let me know which time slot works best for you!


In [6]:
from dotenv import load_dotenv
import os
load_dotenv()
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",
    model_kwargs=dict(temperature=0),
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
    region_name="us-west-2"
    # other params...
)

In [7]:
llm.invoke("hello, who are you?")

AIMessage(content="I'm Claude, an AI assistant created by Anthropic. I aim to be direct and honest in my interactions. How can I help you today?", additional_kwargs={'usage': {'prompt_tokens': 13, 'completion_tokens': 34, 'total_tokens': 47}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20241022-v2:0'}, response_metadata={'usage': {'prompt_tokens': 13, 'completion_tokens': 34, 'total_tokens': 47}, 'stop_reason': 'end_turn', 'model_id': 'anthropic.claude-3-5-sonnet-20241022-v2:0'}, id='run-0c4d197f-f66a-40d1-839e-5b6030f0e78a-0', usage_metadata={'input_tokens': 13, 'output_tokens': 34, 'total_tokens': 47})

In [32]:
def poll_for_messages_from_sender(service, last_check_time, sender_email):
    # Filter messages by sender and time
    query = f'after:{int(last_check_time.timestamp())} from:{sender_email}'
    response = service.users().messages().list(
        userId='me',
        q=query
    ).execute()

    messages = response.get('messages', [])
    for msg in messages:
        msg_id = msg['id']
        msg_details = service.users().messages().get(userId='me', id=msg_id).execute()
        print(f"New message from {sender_email}: {msg_details['snippet']}")  # Process message content here

In [33]:
from datetime import UTC

In [39]:
last_check_time = datetime.now(UTC) - timedelta(days=100)
poll_for_messages_from_sender(email_service, last_check_time, 'transportation@scu.edu')

New message from transportation@scu.edu: Hi SCU Student Community, We are excited to invite you to CSS and P&amp;TS&#39;s 2nd Annual Friendsgiving celebration! Enjoy some delicious food, good company, and a festive atmosphere as we celebrate
New message from transportation@scu.edu: The First-year Winter Quarter Hardship Application will be open starting 11/23/24 - 12/23/24. All qualifying students may apply on their AIMS tile from their MySCU Portal. Please read the information
New message from transportation@scu.edu: Trouble viewing this email? Read it online. Newsletter November 6, 2024 Join the listserv! Spread the word! We are sending announcements through the Announcements list. Please refer colleagues to
New message from transportation@scu.edu: Trouble viewing this email? Read it online. Newsletter October 2024 Parking &amp; Transportation Services (P&amp;TS) Welcome Our New Parking Staff Member, Ted Mielke! Please welcome our new Parking
New message from transportation@scu.edu: 

In [5]:
utc_to_pst(datetime.now(timezone.utc))

datetime.datetime(2024, 11, 16, 13, 24, 44, 662733, tzinfo=datetime.timezone.utc)

In [8]:
now = utc_to_pst(datetime.now(timezone.utc))
now_string = now.strftime("%Y-%m-%d %H:%M:%S")
print(now_string)

2024-11-16 13:27:14



SUBJECT: Tea Invite
BODY: Hi,

I hope this message finds you well! I would like to invite you for a lovely tea gathering on November 15 at 6 PM. It would be a great opportunity to catch up and enjoy some good conversation.

Looking forward to seeing you there!

Best,
Bob



SUBJECT: Merry Christmas!
BODY: Dear Andrew,

As the holiday season approaches, I wanted to take a moment to wish you a Merry Christmas. Here’s a little poem to spread some cheer:

In the glow of twinkling lights,
Joy and laughter fill the nights.
With friends and family gathered near,
Wishing you a Christmas full of cheer.

May your days be merry and bright,
And your heart be warm with delight.

Wishing you all the best this holiday season!

Warm regards,

[Your Name]



SUBJECT: hello
BODY: <div dir="ltr">test</div>




In [22]:
def get_now_string():
    now = utc_to_pst(datetime.now(timezone.utc))
    now_string = now.strftime("%Y-%m-%d %H:%M:%S")
    return now_string

In [23]:
get_now_string()

'2024-11-16 14:02:40'

In [47]:
from langchain_openai.chat_models import ChatOpenAI

classifier_llm = ChatOpenAI(temperature=0)

def classify_email(slot_start, slot_end, subject, body):
    classify_email_prompt = f'''
        You are a specialized email-classification assistant. We have previously
        contacted this person to ask them if they are available
        for a meeting that starts at {slot_start} and ends at {slot_end}.
        The email provided might be a response from the user confirming the meeting, or it could be
        denying the meeting at this time, or it could be something totally unrelated.
        Here is the email subject: {subject}
        Here is the email body: {body}
        Please classify this email, output a 0 if it is a confirmation, 1 if it is denial, or 2 if it is unrelated.
    '''

    response = classifier_llm.invoke(classify_email_prompt)
    return response

In [ ]:
# get emails
format_string = "%Y-%m-%d %H:%M:%S"
last_check_time = datetime.strptime("2024-11-16 16:00:38", format_string)
check_time_utc = pst_to_utc(last_check_time)

query = f'after:{int(last_check_time.timestamp())} from:aanokhin@scu.edu'
response = email_service.users().messages().list(
    userId='me',
    q=query
).execute()

messages = response.get('messages', [])

print("MESSAGES: " + str(len(messages)))

# get email bodies
for message in messages:
    msg_id = message["id"]
    full_email = email_service.users().messages().get(userId='me', id=msg_id, format='full').execute()

    payload = full_email.get('payload', {})
    headers = payload.get('headers', [])

    subject = next((header['value'] for header in headers if header['name'] == 'Subject'), "No Subject")

    # Extract the email body
    body = ""
    parts = payload.get('parts', [])
    for part in parts:
        if part['mimeType'] == 'text/plain':  # Plain text body
            body = part.get('body', {}).get('data', "")
        elif part['mimeType'] == 'text/html':  # HTML body
            body = part.get('body', {}).get('data', "")

    # Decode the body if it's Base64 encoded
    if body:
        body = base64.urlsafe_b64decode(body).decode('utf-8')

    print("MESSAGE: " + str(body))

    print(classify_email("2024-11-25 09:00:00", "2024-11-25 11:00:00", subject, body))

MESSAGES: 2
MESSAGE: <div dir="auto">Yes that meeting time works for me</div>

content='0' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 172, 'total_tokens': 173, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-27b4fc18-953a-4b56-b2e3-ddb2033e5ce3-0' usage_metadata={'input_tokens': 172, 'output_tokens': 1, 'total_tokens': 173, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
MESSAGE: <div dir="auto">Yes that works for me!</div><br><div class="gmail_quote"><div dir="ltr" class="gmail_attr">On Sat, Nov 16, 2024, 4:58 PM  &lt;<a href="mailto:bobbykilkins@gmail.com">bobbykilkins@gmail.com</a>&gt; wr

In [ ]:
async def long_running_task():
        print("Task started...")
        config = {"configurable": {"thread_id": state["email"]}}
        for i in range(20):
            print(f"Task for {state["email"]} is running... {i}")
            print(f"INV {state["email"]} is running... {i}")
            await asyncio.sleep(3)  # Simulates a long-running task
            print(f"INV: {bot.rag_graph.get_state(config=config).values["invitees"]}")
            if bot.rag_graph.get_state(config=config).values["bgtask_stop"]:
                bot.rag_graph.update_state(config=config, values={"bgtask_stop": False})
                break
        return "Task done"



    bg_flag = state["is_bgtask_running"]
    if not bg_flag:
        bg_flag = True
        config = {"configurable": {"thread_id": state["email"]}}
        bot.rag_graph.update_state(config=config, values={"invitees": ["andrew.anokhin@gmail.com"]})
        background_task = asyncio.create_task(long_running_task())
        background_task.add_done_callback(on_task_done)

In [74]:
def create_event(service):
    """Create an event on the user's Google Calendar."""
    event = {
        'summary': 'Sample Event',
        'location': '1234 Main St, Example City',
        'description': 'A description of the event.',
        'start': {
            'dateTime': '2024-11-17T09:00:00-07:00',  # Start time (ISO format)
            'timeZone': 'America/Los_Angeles',
        },
        'end': {
            'dateTime': '2024-11-17T10:00:00-07:00',  # End time (ISO format)
            'timeZone': 'America/Los_Angeles'
        },
        'attendees': [
            {'email': 'alexander.g.anokhin@gmail.com'}
        ],
        'reminders': {
            'useDefault': True
        },
    }

    try:
        event = service.events().insert(calendarId='primary', body=event).execute()
        print(f"Event created: {event.get('htmlLink')}")
    except HttpError as error:
        print(f"An error occurred: {error}")


In [75]:
create_event(calendar_service)

Event created: https://www.google.com/calendar/event?eid=MXFzanNiOW5qOWQwdGs2M2ZrMm9uOGJuanMgYm9iYnlraWxraW5zQG0


In [ ]:

# Authenticate and create an event
service = authenticate()
create_event(service)

In [59]:
from googleapiclient.errors import HttpError


In [ ]:
def create_event(creds):
    client_id = os.environ["GOOGLE_CLIENT_ID"]
    client_secret = os.environ["GOOGLE_CLIENT_SECRET"]
    #creds = Credentials(token=access_token, refresh_token=refresh_token, client_id=client_id, client_secret=client_secret)
    service = build('calendar', 'v3', credentials=creds)

    # create event
    event = {
        'summary': 'Sample Event',
        'location': '1234 Main St, Example City',
        'description': 'A description of the event.',
        'start': {
            'dateTime': '2024-11-17T09:00:00-07:00',  # Start time (ISO format)
            'timeZone': 'America/Los_Angeles',
        },
        'end': {
            'dateTime': '2024-11-17T10:00:00-07:00',  # End time (ISO format)
            'timeZone': 'America/Los_Angeles'
        },
        'attendees': [
            {'email': 'alexander.g.anokhin@gmail.com'}
        ],
        'reminders': {
            'useDefault': True
        }
    }

    try:
        event = service.events().insert(calendarId='primary', body=event).execute()
        print(f"Event created: {event.get('htmlLink')}")
    except HttpError as error:
        print(f"An error occurred: {error}")